In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt




In [2]:
raw = mne.io.read_raw_edf('k3b.gdf', preload=True, stim_channel='auto')

# print(raw.ch_names)

# events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')
# picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')
# tmin, tmax = 1., 4.


# event_id = [ 768, 1791, 2323, 2324, 2325, 2326, 2337, 2339, 3346, 3347, 3348, 3349, 3360, 3362]
# epochs = mne.Epochs(raw, events, None, tmin, tmax, proj=True, picks=picks,
#                 baseline=None, preload=True)




Extracting EDF parameters from k3b.gdf...
GDF file detected
Overlapping events detected. Use find_edf_events for the original events.
Setting channel info structure...
Interpolating stim channel. Events may jitter.
Creating raw.info structure...
Reading 0 ... 986779  =      0.000 ...  3947.116 secs...


<ipython-input-2-3bd60b140d72>:1: RuntimeWarning: Overlapping events detected. Use find_edf_events for the original events.
  raw = mne.io.read_raw_edf('k3b.gdf', preload=True, stim_channel='auto')
<ipython-input-2-3bd60b140d72>:1: RuntimeWarning: Interpolating stim channel. Events may jitter.
  raw = mne.io.read_raw_edf('k3b.gdf', preload=True, stim_channel='auto')


In [3]:
print(raw.ch_names[30])

# 31


In [3]:
events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')

data = raw.get_data()
print(data.shape)


# raw = mne.io.read_raw_edf("k3b.gdf", preload=True)
_, pos, kind, chan, dur = raw.find_edf_events()
kind_hex = np.array([hex(n) for n in kind])

print(pos[1])
print(pos[2])
print(pos[:20])

print(len(kind_hex))
print(len(pos))
sampling_rate = 250


beep_pos = [pos[i] for i in range(len(kind_hex)) if kind_hex[i] in ['0x301','0x302','0x303','0x304']]
labels = [i[4] for i in kind_hex if i in ['0x301','0x302','0x303','0x304']]
print(labels)


def making_epochs(data):
    epochs = []
    for b in beep_pos:
        epochs.append(data[:,b+sampling_rate*1:b+sampling_rate*4])
    return np.asarray(epochs)


epochs = making_epochs(data)
# dif = [int(pos[i]) - int(pos[i-1]) for i in range(len(beep)-1) ]

# print(dif[:100])

1080 events found
Event IDs: [ 768 1791 2323 2324 2325 2326 2337 2339 3346 3347 3348 3349 3360 3362]
(61, 986780)
2924
2924
[ 2424  2924  2924  3174  4984  5484  5484  5734  7544  7544  8044  8044
  8294 10104 10604 10604 10854 12665 13165 13165]
1502
1502
['3', '4', '3', '3', '3', '2', '4', '2', '1', '4', '1', '2', '2', '1', '2', '4', '1', '2', '3', '1', '1', '3', '1', '3', '1', '1', '1', '2', '4', '3', '2', '4', '2', '2', '3', '4', '1', '3', '2', '4', '3', '1', '4', '1', '4', '3', '1', '1', '2', '3', '1', '4', '4', '2', '2', '3', '4', '2', '1', '4', '4', '3', '1', '3', '4', '3', '3', '3', '1', '4', '2', '4', '4', '2', '1', '1', '4', '1', '3', '2', '4', '2', '1', '4', '2', '2', '4', '3', '1', '2', '2', '4', '2', '3', '4', '4', '3', '3', '4', '1', '4', '2', '3', '2', '1', '3', '4', '1', '2', '1', '3', '1', '3', '4', '3', '3', '1', '4', '3', '4', '4', '2', '2', '1', '2', '4', '4', '3', '3', '1', '4', '1', '3', '1', '2', '2', '3', '3', '3', '1', '1', '2', '2', '2', '1', '1', '2', '2', '1

In [50]:
print(epochs.shape)



(180, 61, 750)


In [53]:
import sklearn 
import numpy as np
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = epochs

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(y_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    TSclassifier = pyriemann.classification.TSclassifier(metric='riemann')
    TSclassifier.fit(cov_X_train, y_train)
    
    
    y_predict = TSclassifier.predict(cov_X_test)
    print(sklearn.metrics.accuracy_score(TSclassifier.predict(cov_X_train), y_train))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
Classification accuracy: 0.913889 / Chance level: 0.250000


In [4]:
import pywt

testing_epochs = []
single_epoch = []
samples, chan_num, timesteps = epochs.shape
epochs_power = epochs**2   
# for s in range(samples):
#     single_epoch = []
#     single_epoch.append(pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[0])
#     single_epoch.append(pywt.wavedec(epochs_power[s,30,:], 'db4', level=4)[0])
#     single_epoch.append(pywt.wavedec(epochs_power[s,33,:], 'db4', level=4)[0])
#     testing_epochs.append(np.asarray(single_epoch))
    
    
for s in range(samples):
    single_epoch = []
    single_epoch = np.asarray(single_epoch).reshape((1,-1))
    for xi in pywt.wavedec(epochs_power[s,27,:], 'db4', level=4):
        single_epoch = np.concatenate((single_epoch,np.asarray(xi).reshape((1,-1))), axis = 1)

    testing_epochs.append(single_epoch)
  

testing_epochs = np.asarray(testing_epochs)
print(testing_epochs.shape)
#9540

print(np.count_nonzero(abs(testing_epochs) > 1.0056865244277509e-9))
print(np.min(abs(testing_epochs)))


testing_epochs = testing_epochs.reshape((180, 775))

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(testing_epochs)
# print(scaler.mean_)
testing_epochs = scaler.transform(testing_epochs)


(180, 1, 775)
91
1.0490812995593966e-16


In [89]:
# np.concatenate(pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[0], pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[1])

y = []
y = np.asarray(y).reshape((1,-1))
for xi in pywt.wavedec(epochs_power[s,27,:], 'db4', level=4):
    y = np.concatenate((y,np.asarray(xi).reshape((1,-1))), axis = 1)
    
# print(pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[0].shape)
#378 192 99  53  53
print(y.shape)


(1, 775)


In [126]:
weights, biases = model.layers[0].get_weights()
print(np.asarray(weights)[:,0])

[-0.02026197  0.01008344  0.0281049  -0.00588483 -0.02329246 -0.01131399
  0.02286122  0.02807407 -0.02480852  0.02144835 -0.03157116  0.02203869
  0.02432689 -0.03360921 -0.01371121  0.0332848   0.02939171 -0.01283365
  0.00913258  0.03174998 -0.01887629  0.02043019  0.03358698 -0.02914592
 -0.01815564  0.01274412  0.02527639  0.0003182   0.01464843  0.00697661
  0.0038463   0.02804249  0.0028288  -0.00961178  0.03710476  0.01277776
 -0.01071792  0.00165564  0.02713612 -0.01148787 -0.00614739 -0.00524904
  0.01256036  0.01731174  0.01820919  0.02143856 -0.00090359 -0.00530857
  0.03376178 -0.01795817 -0.00377397  0.00915935 -0.01166671  0.00854721
 -0.00649501  0.02719112 -0.00645062  0.0202097   0.00469979  0.02829024
  0.00837883 -0.02156244 -0.01468641 -0.02569531 -0.01841003 -0.00140771
 -0.03338108 -0.0058276   0.02066448  0.02221901 -0.00117241 -0.00197715
  0.03047566  0.01854157  0.03394988  0.00581654 -0.02598906 -0.00897026
  0.02640313 -0.02844475  0.0132376   0.03251541  0

In [143]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras import backend as K
import keras
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft
from keras import optimizers
from keras import regularizers

K.clear_session()

# testing_epochs = testing_epochs.reshape((180,1, 775))


class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

history = LossHistory()


from keras.callbacks import EarlyStopping
stop_here_please = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)


testing_epochs = np.asarray(testing_epochs)

# testing_epochs = testing_epochs.swapaxes(1,2)

# time_steps = 53
# n_features = 3

time_steps = 775
n_features = 1



cv = ShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
scores = []
testing_epochs_ANN = testing_epochs.reshape(testing_epochs.shape[0], -1)
cvscores = []

for train_idx, test_idx in cv.split(testing_epochs):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = testing_epochs[train_idx]
    X_test = testing_epochs[test_idx]
    
    

    model = Sequential()
    model.add(Dense(units = 5, input_dim=775, activation="relu", \
                    kernel_initializer="uniform", kernel_regularizer=regularizers.l2(0.1)))
    model.add(Dropout(0.2))
    model.add(Dense(units = 5, init='uniform', activation='sigmoid'))
    model.compile( loss = "categorical_crossentropy", 
#                optimizer = keras.optimizers.SGD(lr=0.01, clipvalue=0.5), 
                optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0),
               metrics=['accuracy']
             )
    
  #  batch_size cannot be 1, at least contain all the classes for cross-entropy
    model.fit(X_train, keras.utils.np_utils.to_categorical(y_train), batch_size=18, epochs=100, callbacks=[stop_here_please])

    print(model.predict_classes(X_train))
    print(y_train)


#     print(history.losses)

    

#     model = Sequential()
#     model.add(LSTM(3, return_sequences=False, input_shape=(time_steps, n_features)))
#     model.add(Dropout(0.2))
#     model.add(LSTM(10, return_sequences=False, input_shape=(time_steps, n_features)))
#     model.add(Dense(50, activation='sigmoid'))
#     model.add(Dense(5, activation='sigmoid'))
#     model.compile(loss="categorical_crossentropy",
#                   optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), 
#                   metrics=['accuracy'])
#     model.fit(X_train, keras.utils.np_utils.to_categorical(y_train), batch_size=36, epochs=2000, callbacks=[history])



    scores = model.evaluate(X_test, keras.utils.np_utils.to_categorical(y_test), verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
    
    
print(np.mean(cvscores))







C:\Users\kun\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:59: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=5, activation="sigmoid", kernel_initializer="uniform")`


Epoch 1/100
144/144 [==============================] - 0s 968us/step - loss: 1.8218 - acc: 0.2292
Epoch 2/100
144/144 [==============================] - 0s 56us/step - loss: 1.7073 - acc: 0.2361
Epoch 3/100
144/144 [==============================] - 0s 53us/step - loss: 1.6562 - acc: 0.3611
Epoch 4/100
144/144 [==============================] - 0s 60us/step - loss: 1.6274 - acc: 0.4236
Epoch 5/100
144/144 [==============================] - 0s 83us/step - loss: 1.6100 - acc: 0.5208
Epoch 6/100
144/144 [==============================] - 0s 99us/step - loss: 1.6005 - acc: 0.5208
Epoch 7/100
144/144 [==============================] - 0s 111us/step - loss: 1.5916 - acc: 0.5208
Epoch 8/100
144/144 [==============================] - 0s 109us/step - loss: 1.5850 - acc: 0.5347
Epoch 9/100
144/144 [==============================] - 0s 83us/step - loss: 1.5744 - acc: 0.6181
Epoch 10/100
144/144 [==============================] - 0s 111us/step - loss: 1.5644 - acc: 0.6319
Epoch 11/100
144/144 [===

C:\Users\kun\Anaconda2\envs\py36\lib\site-packages\keras\callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


144/144 [==============================] - 0s 56us/step - loss: 1.5066 - acc: 0.6250
Epoch 16/100
144/144 [==============================] - 0s 83us/step - loss: 1.4936 - acc: 0.5556
Epoch 17/100
144/144 [==============================] - 0s 106us/step - loss: 1.4839 - acc: 0.6181
Epoch 18/100
144/144 [==============================] - 0s 56us/step - loss: 1.4691 - acc: 0.6250
Epoch 19/100
144/144 [==============================] - 0s 83us/step - loss: 1.4681 - acc: 0.6042
Epoch 20/100
144/144 [==============================] - 0s 56us/step - loss: 1.4618 - acc: 0.5556
Epoch 21/100
144/144 [==============================] - 0s 65us/step - loss: 1.4351 - acc: 0.6250
Epoch 22/100
144/144 [==============================] - 0s 56us/step - loss: 1.4332 - acc: 0.5903
Epoch 23/100
144/144 [==============================] - 0s 83us/step - loss: 1.4075 - acc: 0.6250
Epoch 24/100
144/144 [==============================] - 0s 56us/step - loss: 1.4043 - acc: 0.5903
Epoch 25/100
144/144 [==========

In [85]:
print(model.predict_classes(X_test))
print(y_test)



[2 1 1 4 3 2 2 2 1 3 4 1 4 2 4 2 4 2 2 2 1 4 2 1 4 1 4 2 2 2 1 3 4 4 3 3]
['1' '4' '3' '3' '1' '4' '1' '1' '4' '3' '3' '2' '4' '2' '3' '1' '4' '4'
 '3' '4' '3' '4' '2' '2' '4' '4' '4' '3' '1' '2' '1' '3' '1' '4' '3' '3']


In [ ]:
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
scaler = StandardScaler()
print(scaler.fit(data))
StandardScaler(copy=True, with_mean=True, with_std=True)
print(scaler.mean_)
print(scaler.transform(data))


Baseline PCA + Logistic regression model

In [55]:
import sklearn
from sklearn.decomposition import PCA
from sklearn import linear_model, decomposition, datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit  


samples = []
for s in range(epochs.shape[0]):
    X = epochs[s,:,:]
    pca = PCA(n_components=20)
    pca.fit(X)
    pca_com = pca.components_.flatten()
    samples.append(pca_com)
samples = np.asarray(samples)
# print(samples.shape)
# logistic = linear_model.LogisticRegression()
# logistic.fit(samples, np.asarray(labels))
# y_pred = logistic.predict(samples)


scores = []
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
for train_idx, test_idx in cv.split(samples):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = samples[train_idx]
    X_test = samples[test_idx]


    
    logistic = linear_model.LogisticRegression(C = 1e-5)
    logistic.fit(X_train, y_train)
    

    
    y_predict = logistic.predict(X_test)
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

(180, 61, 750)
Classification accuracy: 0.166667 / Chance level: 0.250000
